In [20]:
#import splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


In [9]:
#set up executable paths and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\purvi\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [10]:
#visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)

#optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [11]:
#set up HTML parser
html = browser.html
bs = soup(html, 'html.parser')
slide_elem = bs.select_one('div.list_text')

In [12]:
#begin scraping article titles
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's New Mars Rover Is Ready for Space Lasers</div>

In [13]:
#use parent element to find first 'a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's New Mars Rover Is Ready for Space Lasers"

In [14]:
#use the parent element to find paragraph text of article 
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Perseverance is one of a few Mars spacecraft carrying laser retroreflectors. The devices could provide new science and safer Mars landings in the future.'

###Featured Images

In [15]:
#visit url
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [16]:
#find and click full size image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [17]:
#parse resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [18]:
#find relative image url; only want most recent image loaded
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [19]:
#add base url to code to retrieve images
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [21]:
#establish dataframe from html table using read_html pandas command
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [22]:
#convert df to html data table
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [23]:
#end session
browser.quit()